In [38]:
!pip3 install bokeh

In [39]:
from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import row
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool

output_notebook()

Loading BokehJS ...

In [40]:
import pandas as pd

def get_usable_features():
    """Return 1991 fields and 2016 fields usable for cross-year comparison."""
    
    field_name_data = pd.read_csv('Data_Mining_Names.csv')
    
    features_1991 = []
    features_2016 = []

    for older, newer in zip(field_name_data['dataset_1991'], field_name_data['dataset_2016']):
        if older != '-':
            features_1991.append(older)
            features_2016.append(newer)
    
    return features_1991, features_2016


features_1991, features_2016 = get_usable_features()

In [41]:
import pandas as pd

fields = features_1991

dataset = pd.read_csv("pumf-95M0007-E-1991-individuals_F1.csv", usecols=fields)
dataset.head()

,PROVP,CMAPUMFP,HTYPEP,UNITSP,HHINCP,EFSIZEP,CFSTATP,CFSIZEP,CFINCP,AGEP,...,WKSWKP,TOTINCP,INCSTP,TENURP,RCONDP,PRMAINP,ROOMP,CONDWELP,VALUEP,WEIGHTP
0,10,999,3,4,17,4,4,4,17,34,...,14,12424,1,1,2,2,10,1,40000,33.3333
1,10,999,3,4,17,4,7,4,17,3,...,99,9999999,1,1,2,2,10,1,40000,33.3333
2,10,999,3,5,21,5,1,5,21,44,...,20,30000,1,1,2,1,9,1,40000,33.3333
3,10,999,3,5,21,5,7,5,21,12,...,99,9999999,1,1,2,2,9,1,40000,33.3333
4,10,999,2,4,17,4,1,2,8,65,...,99,7639,1,1,2,1,8,2,20000,33.3333


In [42]:
from math import isnan

def get_language_mappings():
    """Get language mapping dicts, separated into subdicts by feature name."""

    hlano = {}
    mtnno = {}
    hlnp = {}
    mtnp = {}
    classes = set()

    language_mappings = pd.read_csv('Language_Mappings.csv')
    
    for row in list(language_mappings.itertuples()):
        # print(row[1])
        # print(row[3])
        # print(row[5])
        # print(row[7])
        # print(row[9])

        classes.add(row[9])
        
        if not isnan(row[1]):
            hlano[int(row[1])] = row[9]
        if not isnan(row[3]):
            mtnno[int(row[3])] = row[9]
        if not isnan(row[5]):
            hlnp[int(row[5])] = row[9]
        if not isnan(row[7]):
            mtnp[int(row[7])] = row[9]

    return {
        'hlano': hlano,
        'mtnno': mtnno,
        'hlnp': hlnp,
        'mtnp': mtnp,
        'classes': list(classes)
    }


mappings = get_language_mappings()

motherTongueDict = mappings['mtnp']
homeLanguageDict = mappings['hlnp']

In [43]:
motherTongueDict['1'] = 'English_Only'
motherTongueDict['2'] = 'French_Only'
motherTongueDict['3'] = 'English_And_French'

homeLanguageDict['1'] = 'English_Only'
homeLanguageDict['2'] = 'French_Only'
homeLanguageDict['3'] = 'English_And_French'

In [44]:
dataset = dataset[dataset.HLNP != 88]
dataset = dataset[dataset.MTNP != 88]
# class label: home language part A - first language write in component
homeLang = dataset['HLNP']
# class label: mother tongue part A - first language write in component
motherTongue = dataset['MTNP']
fields.remove('HLNP')
fields.remove('MTNP')
homeLanguageMapped = homeLang.apply(lambda x: homeLanguageDict[x])
motherTongueMapped = motherTongue.apply(lambda x: motherTongueDict[x])
languageShift = homeLanguageMapped.ne(motherTongueMapped)

fields.remove('WEIGHTP')
x = dataset[fields]
weights = dataset["WEIGHTP"]

In [45]:
homeLanguageMapped

0         Official_Languages
1         Official_Languages
2         Official_Languages
3         Official_Languages
4         Official_Languages
                 ...        
809649    Official_Languages
809650    Official_Languages
809651    Official_Languages
809652    Official_Languages
809653    Official_Languages
Name: HLNP, Length: 809654, dtype: object

In [46]:
motherTongueMapped

0         Official_Languages
1         Official_Languages
2         Official_Languages
3         Official_Languages
4         Official_Languages
                 ...        
809649    Official_Languages
809650    Official_Languages
809651    Official_Languages
809652    Official_Languages
809653            Aboriginal
Name: MTNP, Length: 809654, dtype: object

In [47]:
languageShift

0         False
1         False
2         False
3         False
4         False
          ...  
809649    False
809650    False
809651    False
809652    False
809653     True
Length: 809654, dtype: bool

In [48]:
x

,ABETHNCP,AGEP,BNFNMEMP,CFINCP,CFSIZEP,CFSTATP,CITIZENP,CMAPUMFP,CONDWELP,COWP,...,RCONDP,REGINP,ROOMP,SECGRADP,SEXP,TENURP,TOTINCP,UNITSP,VALUEP,WKSWKP
0,5,34,2,17,4,4,1,999,1,1,...,2,2,10,2,1,1,12424,4,40000,14
1,5,3,2,17,4,7,1,999,1,9,...,2,2,10,9,1,1,9999999,4,40000,99
2,5,44,2,21,5,1,1,999,1,1,...,2,2,9,5,2,1,30000,5,40000,20
3,5,12,2,21,5,7,1,999,1,9,...,2,2,9,9,1,1,9999999,5,40000,99
4,5,65,2,8,2,1,1,999,2,9,...,2,2,8,1,2,1,7639,4,20000,99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
809649,2,2,2,99,1,11,1,999,2,9,...,2,2,5,9,1,1,9999999,7,75000,99
809650,2,24,2,12,4,4,1,999,3,9,...,9,2,5,1,1,2,5520,4,999999,99
809651,2,5,2,12,4,7,1,999,3,9,...,9,2,5,9,2,2,9999999,4,999999,99
809652,2,1,2,12,4,7,1,999,3,9,...,9,2,5,9,2,2,9999999,4,999999,99


In [49]:
# discretization

def discretify(dataset, quantiles=10, target_fields=None,
               invalid_quantile = 88):
    """Convert target fields in dataset to quantile categories."""
    
    target_fields = target_fields if target_fields else ['AGEP', 'WKSWKP', 'SHELCO', 'MrkInc', 'TotInc_AT',
                                                         'TotInc', 'VALUE', 'TOTINCP', 'VALUEP']
    invalid_values = {
        'AGEP': [98],
        'WKSWKP': [99],
        'SHELCO': [],
        'MrkInc': [88888888, 99999999],
        'TotInc': [88888888, 99999999],
        'TotInc_AT': [88888888, 99999999],
        'VALUE': [88888888, 99999999],
        'VALUEP': [999999],
        'TOTINCP': [9999999]
    }
    
    dataset_copy = dataset.copy()

    for continuous in target_fields:
        if continuous in dataset_copy:
            print(continuous)
            values = []
            quantile_values = pd.qcut(dataset_copy[continuous].rank(method='first'), quantiles, False)

            for value, quantile in zip(dataset_copy[continuous], quantile_values):
                if value in invalid_values[continuous]:
                    values.append(invalid_quantile)
                else:
                    values.append(quantile)

            dataset_copy[continuous] = values
    
    return dataset_copy

x = discretify(x)

x

AGEP
WKSWKP
TOTINCP
VALUEP


,ABETHNCP,AGEP,BNFNMEMP,CFINCP,CFSIZEP,CFSTATP,CITIZENP,CMAPUMFP,CONDWELP,COWP,...,RCONDP,REGINP,ROOMP,SECGRADP,SEXP,TENURP,TOTINCP,UNITSP,VALUEP,WKSWKP
0,5,5,2,17,4,4,1,999,1,1,...,2,2,10,2,1,1,3,4,0,0
1,5,0,2,17,4,7,1,999,1,9,...,2,2,10,9,1,1,88,4,0,88
2,5,6,2,21,5,1,1,999,1,1,...,2,2,9,5,2,1,5,5,0,0
3,5,1,2,21,5,7,1,999,1,9,...,2,2,9,9,1,1,88,5,0,88
4,5,8,2,8,2,1,1,999,2,9,...,2,2,8,1,2,1,2,4,0,88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
809649,2,0,2,99,1,11,1,999,2,9,...,2,2,5,9,1,1,88,7,1,88
809650,2,3,2,12,4,4,1,999,3,9,...,9,2,5,1,1,2,1,4,88,88
809651,2,0,2,12,4,7,1,999,3,9,...,9,2,5,9,2,2,88,4,88,88
809652,2,0,2,12,4,7,1,999,3,9,...,9,2,5,9,2,2,88,4,88,88


In [50]:
weights

0         33.3333
1         33.3333
2         33.3333
3         33.3333
4         33.3333
           ...   
809649    33.3333
809650    33.3333
809651    33.3333
809652    33.3333
809653    33.3333
Name: WEIGHTP, Length: 809654, dtype: float64

In [51]:
!pip3 install sklearn

In [52]:
from sklearn.ensemble import RandomForestClassifier

In [53]:
classifier = RandomForestClassifier(n_estimators=20, random_state=0)

In [54]:
from sklearn.model_selection import train_test_split
weights_train, weights_test, x_train, x_test, languageShift_train, languageShift_test = train_test_split(weights, x, languageShift, random_state=69)

In [55]:
classifier.fit(x_train, languageShift_train, weights_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=20,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [56]:
languageShift_pred = classifier.predict(x_test)

languageShift_probs = classifier.predict_proba(x_test)[:, 1]

In [57]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

The Confusion Matrix indicates, given the predicted language (row names), for how many records did the algorith match the actual language. The classification report and accuracy score are other ways to evaluate how well the classification algorithm performed.

In [58]:
confusionMatrixLanguageShift = pd.DataFrame(
    confusion_matrix(languageShift_test,languageShift_pred), 
    index=['False', 'True'], 
    columns=['False', 'True'], 
)

In [59]:
print(confusionMatrixLanguageShift)
print(classification_report(languageShift_test,languageShift_pred,target_names=['False', 'True']))
print(accuracy_score(languageShift_test, languageShift_pred))

        False  True
False  184802  2389
True    10641  4582
              precision    recall  f1-score   support

       False       0.95      0.99      0.97    187191
        True       0.66      0.30      0.41     15223

    accuracy                           0.94    202414
   macro avg       0.80      0.64      0.69    202414
weighted avg       0.92      0.94      0.92    202414

0.9356269823233571


In [60]:
from sklearn.metrics import roc_auc_score, roc_curve

auc_score = roc_auc_score(languageShift_test, languageShift_probs)
auc_score

0.9242720821273154

In [61]:
base_fpr, base_tpr, _ = roc_curve(languageShift_test, [1 for _ in range(len(languageShift_test))])
model_fpr, model_tpr, _ = roc_curve(languageShift_test, languageShift_probs)

# Generate a figure container
plot = figure(plot_height=400,
              plot_width=400,
              title='ROC AUC Score',
              x_axis_label='False Positive Rate',
              y_axis_label='True Positive Rate',
              x_range=[0, 1],
              y_range=[0, 1])

plot.line(model_fpr, model_tpr, line_width=3, color='red', legend_label='Random Forest')
plot.line(base_fpr, base_tpr, line_width=3, line_dash='dashed', legend_label='Random')

plot.title.align = 'center'
plot.legend.location = 'bottom_right'

show(plot)

In [62]:
!pip install tabulate

In [69]:
def get_unified_feature_dicts():
    """Return 1991-2016 unified verbose field name dict and 1991-2016 unified verbose field description dict."""
    
    field_name_data = pd.read_csv('Data_Mining_Names.csv')
    
    data_types = field_name_data['Data_Type']
    meanings = field_name_data['Meaning']

    descriptions = []

    for types, meaning in zip(data_types, meanings):
        descriptions.append(types + ': ' + meaning)

    description_dict = dict(zip(field_name_data['dataset_2016'], descriptions))
    name_dict = dict(zip(field_name_data['dataset_2016'], field_name_data['Definitive_Name']))
    
    description_dict.update(dict(zip(field_name_data['dataset_1991'], descriptions)))
    name_dict.update(dict(zip(field_name_data['dataset_1991'], field_name_data['Definitive_Name'])))
    
    del description_dict['-']
    del name_dict['-']
    
    return description_dict, name_dict


description_dict, name_dict = get_unified_feature_dicts()

descriptions = [description_dict[feature] if feature in description_dict else feature for feature in x.columns]
true_names = [name_dict[feature] if feature in name_dict else feature for feature in x.columns]

In [70]:
from tabulate import tabulate
importances = classifier.feature_importances_
headers = ["name", "score", "verbose_name", "description"]
values = sorted(zip(x.columns, classifier.feature_importances_, true_names, descriptions), key=lambda x: x[1] * -1)
print(tabulate(values, headers, tablefmt="plain"))

name            score  verbose_name                          description
ETHNICRP  0.173528     Ethnic_Origin                         Categorical: Ethnic origin
POBP      0.0840865    Place_of_Birth                        Categorical: Place of birth
HHINCP    0.0512148    Household_Income_Bracket              Categorical: Household income bracket
ROOMP     0.0472156    Household_Room_Count                  Discrete (0-11+): Number of rooms in household
CFINCP    0.0457294    Census_Family_Income_Bracket          Categorical: Census family income bracket
IMMPOPP   0.0419252    Immigrant_Status                      Categorical: Immigration status (e.g. immigrant vs non)
CMAPUMFP  0.0366023    Metro_Area_Current_Residence          Categorical: Area of current residence
CITIZENP  0.0361436    Canadian_Citizenship                  Categorical: Canadian citizenship status
AGEP      0.0343823    Age_Bracket                           Categorical: Age bracket
TOTINCP   0.0335527    Total_Person

In [71]:
from sklearn.tree import export_graphviz
estimator = classifier.estimators_[5]
# Creates dot file
export_graphviz(estimator, 
                out_file='randomForestTree1991.dot', 
                feature_names = list(x.columns),
                class_names = ['False', 'True'],
                rounded = True, proportion = False, 
                precision = 2, filled = True,
                max_depth=5)

In [72]:
!pip install bokeh

In [73]:
import numpy as np

In [74]:
values = sorted(zip(x.columns, classifier.feature_importances_), key=lambda x: x[1] * -1)

features = pd.DataFrame(values, columns=['Feature', 'Gini'])
p = figure(x_range=features['Feature'], plot_height=500, plot_width=1200, title="Feature Importances",
           toolbar_location=None, x_axis_label='Feature', y_axis_label='Gini index', tools="")
p.vbar(x='Feature', top='Gini', width=0.9, source=ColumnDataSource(data=features))

p.xaxis.major_label_orientation = np.pi/4
p.xgrid.grid_line_color = None
p.y_range.start = 0

p.add_tools(HoverTool(tooltips=[('Feature', '@Feature'),
                                ('Gini index', '@Gini')]))

show(p)